In [5]:
import pandas as pd
import numpy as np
import data 
from bdint.models import LinearRegression
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt

robustscaler = False 
train_df = data.get_train_df()
test_df = data.get_test_df()

model = LinearRegression(train=train_df, test=test_df, skewness_threshold=0.7, alpha = 1, regression_type="Ridge")

df = model.train
df = df.astype('float64')
norm = 'Normalizaiton'
if robustscaler==True:
    norm = 'RobustScaler'
    robust_scl = RobustScaler()
    robust_scl.fit(df)
    df_scaled = robust_scl.transform(df)
    model.train = pd.DataFrame(data = df_scaled, columns = df.columns, index = df.index)
else:   
    normalized_df=(df-df.min())/(df.max()-df.min())
    normalized_df.fillna(0, inplace=True) # fix case when max is 0 (all False)
    # normalized_df['SalePrice'] = train_df['SalePrice']
    model.train = normalized_df

model.learn(train_x=model.train.drop(columns=["SalePrice"], inplace=False), train_y=pd.DataFrame(model.train["SalePrice"]))

coefficients = model.model.coef_
index_highest_coefficients = np.argsort(abs(coefficients))[0][-20:][::-1]
feature_names = model.train.columns[index_highest_coefficients]

plt.bar(model.train.columns[index_highest_coefficients], coefficients[0][index_highest_coefficients])
plt.axhline(y=0, color='b', linestyle='-')
plt.xticks(rotation='vertical')
plt.ylabel('coefficient value')
plt.xlabel('Feature')
plt.title(f'Highest coefficients of {model.name} Regression with {norm}')
plt.savefig(f"vizualization/regression_coefficients_{model.name}_{norm}.png")
plt.show()


MISSING COLUMNS: ['SalePrice']


/home/ewa/.cache/pypoetry/virtualenvs/bdint-SOv9kUne-py3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AttributeError: coef_ is only available when using a linear kernel